# Importing the packages

In [1]:
import tqdm
import optuna
import mlflow
import dagshub
import numpy as np
import pandas as pd

from xgboost import XGBRegressor
from sklearn import set_config
from sklearn.svm import SVR
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder
from sklearn.linear_model import LinearRegression, Ridge
from sklearn.metrics import mean_absolute_percentage_error
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor

set_config(transform_output="pandas")

In [2]:
# MLFlow and DagsHub

mlflow.set_tracking_uri("https://dagshub.com/SushrutGaikwad/taxi-demand-prediction.mlflow")
dagshub.init(repo_owner='SushrutGaikwad', repo_name='taxi-demand-prediction', mlflow=True)

Accessing as SushrutGaikwad

Initialized MLflow to track repo "SushrutGaikwad/taxi-demand-prediction"

Repository SushrutGaikwad/taxi-demand-prediction initialized!

# Loading the data

In [3]:
TRAIN_DATA_PATH = "../data/processed/train.csv"
TEST_DATA_PATH = "../data/processed/test.csv"

train_df = pd.read_csv(TRAIN_DATA_PATH, parse_dates=["tpep_pickup_datetime"]).set_index("tpep_pickup_datetime")
test_df = pd.read_csv(TEST_DATA_PATH, parse_dates=["tpep_pickup_datetime"]).set_index("tpep_pickup_datetime")

In [4]:
train_df

,lag_1,lag_2,lag_3,lag_4,region,total_pickups,avg_pickups,day_of_week
tpep_pickup_datetime,,,,,,,,
2016-01-01 01:00:00,160.0,149.0,120.0,58.0,0,187,161.0,4
2016-01-01 01:15:00,187.0,160.0,149.0,120.0,0,194,175.0,4
2016-01-01 01:30:00,194.0,187.0,160.0,149.0,0,180,177.0,4
2016-01-01 01:45:00,180.0,194.0,187.0,160.0,0,197,185.0,4
2016-01-01 02:00:00,197.0,180.0,194.0,187.0,0,185,185.0,4
...,...,...,...,...,...,...,...,...
2016-02-29 22:45:00,15.0,9.0,11.0,11.0,29,12,12.0,0
2016-02-29 23:00:00,12.0,15.0,9.0,11.0,29,17,14.0,0
2016-02-29 23:15:00,17.0,12.0,15.0,9.0,29,15,14.0,0


In [5]:
test_df

,lag_1,lag_2,lag_3,lag_4,region,total_pickups,avg_pickups,day_of_week
tpep_pickup_datetime,,,,,,,,
2016-03-01 00:00:00,36.0,44.0,31.0,29.0,0,41,39.0,1
2016-03-01 00:15:00,41.0,36.0,44.0,31.0,0,35,37.0,1
2016-03-01 00:30:00,35.0,41.0,36.0,44.0,0,47,41.0,1
2016-03-01 00:45:00,47.0,35.0,41.0,36.0,0,34,38.0,1
2016-03-01 01:00:00,34.0,47.0,35.0,41.0,0,30,35.0,1
...,...,...,...,...,...,...,...,...
2016-03-31 22:45:00,22.0,14.0,15.0,13.0,29,14,16.0,3
2016-03-31 23:00:00,14.0,22.0,14.0,15.0,29,17,16.0,3
2016-03-31 23:15:00,17.0,14.0,22.0,14.0,29,18,17.0,3


## Checking for missing values

In [6]:
train_df.isna().sum()

lag_1            0
lag_2            0
lag_3            0
lag_4            0
region           0
total_pickups    0
avg_pickups      0
day_of_week      0
dtype: int64

In [7]:
test_df.isna().sum()

lag_1            0
lag_2            0
lag_3            0
lag_4            0
region           0
total_pickups    0
avg_pickups      0
day_of_week      0
dtype: int64

# Input-output split

In [8]:
X_train = train_df.drop(columns=["total_pickups"])
y_train = train_df["total_pickups"]

X_test = test_df.drop(columns=["total_pickups"])
y_test = test_df["total_pickups"]

In [9]:
X_train.head()

,lag_1,lag_2,lag_3,lag_4,region,avg_pickups,day_of_week
tpep_pickup_datetime,,,,,,,
2016-01-01 01:00:00,160.0,149.0,120.0,58.0,0,161.0,4
2016-01-01 01:15:00,187.0,160.0,149.0,120.0,0,175.0,4
2016-01-01 01:30:00,194.0,187.0,160.0,149.0,0,177.0,4
2016-01-01 01:45:00,180.0,194.0,187.0,160.0,0,185.0,4
2016-01-01 02:00:00,197.0,180.0,194.0,187.0,0,185.0,4


In [10]:
y_train.head()

tpep_pickup_datetime
2016-01-01 01:00:00    187
2016-01-01 01:15:00    194
2016-01-01 01:30:00    180
2016-01-01 01:45:00    197
2016-01-01 02:00:00    185
Name: total_pickups, dtype: int64

In [11]:
X_test.head()

,lag_1,lag_2,lag_3,lag_4,region,avg_pickups,day_of_week
tpep_pickup_datetime,,,,,,,
2016-03-01 00:00:00,36.0,44.0,31.0,29.0,0,39.0,1
2016-03-01 00:15:00,41.0,36.0,44.0,31.0,0,37.0,1
2016-03-01 00:30:00,35.0,41.0,36.0,44.0,0,41.0,1
2016-03-01 00:45:00,47.0,35.0,41.0,36.0,0,38.0,1
2016-03-01 01:00:00,34.0,47.0,35.0,41.0,0,35.0,1


In [12]:
y_test.head()

tpep_pickup_datetime
2016-03-01 00:00:00    41
2016-03-01 00:15:00    35
2016-03-01 00:30:00    47
2016-03-01 00:45:00    34
2016-03-01 01:00:00    30
Name: total_pickups, dtype: int64

# Encoding

In [13]:
encoder = ColumnTransformer(
    [
        ("ohe", OneHotEncoder(drop="first", sparse_output=False), ["region", "day_of_week"]),
    ],
    remainder="passthrough",
    n_jobs=-1,
    force_int_remainder_cols=False,
)
encoder

ColumnTransformer(force_int_remainder_cols=False, n_jobs=-1,
                  remainder='passthrough',
                  transformers=[('ohe',
                                 OneHotEncoder(drop='first',
                                               sparse_output=False),
                                 ['region', 'day_of_week'])])

In [14]:
X_train_encoded = encoder.fit_transform(X_train)
X_test_encoded = encoder.transform(X_test)

# Model Selection & Hyperparameter Tuning

In [15]:
# Experiment name

mlflow.set_experiment("model_selection")

2025/03/21 23:38:28 INFO mlflow.tracking.fluent: Experiment with name 'model_selection' does not exist. Creating a new experiment.


<Experiment: artifact_location='mlflow-artifacts:/e7025ecf811645d197bc7740aa83c0f9', creation_time=1742614708910, experiment_id='0', last_update_time=1742614708910, lifecycle_stage='active', name='model_selection', tags={}>

In [16]:
def objective(trial):
    # Start with the child run
    with mlflow.start_run(nested=True) as child:
        # Models and their hyperparameter search space
        list_of_models = ["LR", "RF", "GBR", "XGBR"]
        model_name = trial.suggest_categorical("model_name", list_of_models)

        # Going over the models and their hyperparameters
        if model_name == "LR":
            model = LinearRegression()

        elif model_name == "RF":
            n_estimators_rf = trial.suggest_int("n_estimators_rf", 10, 100, step=10)
            max_depth_rf = trial.suggest_int("max_depth_rf", 3, 10)
            model = RandomForestRegressor(
                n_estimators=n_estimators_rf,
                max_depth=max_depth_rf,
                random_state=42,
                n_jobs=-1
            )

        elif model_name == "GBR":
            n_estimators_gb = trial.suggest_int("n_estimators_gb", 10, 100, step=10)
            learning_rate_gb = trial.suggest_float("learning_rate_gb", 1e-4, 1e-1, log=True)
            model = GradientBoostingRegressor(
                n_estimators=n_estimators_gb,
                learning_rate=learning_rate_gb,
                random_state=42
            )

        elif model_name == "XGBR":
            n_estimators_xgb = trial.suggest_int("n_estimators_xgb", 10, 100, step=10)
            learning_rate_xgb = trial.suggest_float("learning_rate_xgb", 1e-4, 1e-1, log=True)
            max_depth_xgb = trial.suggest_int("max_depth_xgb", 3, 10)
            model = XGBRegressor(
                n_estimators=n_estimators_xgb,
                learning_rate=learning_rate_xgb,
                max_depth=max_depth_xgb
            )

        # Logging the model name
        mlflow.log_param("model_name", model_name)

        # Logging the model parameters
        mlflow.log_params(model.get_params())

        # Training the model
        model.fit(X_train_encoded, y_train)

        # Getting the predictions
        y_pred = model.predict(X_test_encoded)

        # Calculating MAPE
        mape = mean_absolute_percentage_error(y_true=y_test, y_pred=y_pred)

        # Logging MAPE
        mlflow.log_metric("mape", mape)

        return mape

In [17]:
# Optimizing the objective function to get the best model

with mlflow.start_run(run_name="best_model", nested=True) as parent:
    # Creating a study object
    study = optuna.create_study(study_name="model_selection", direction="minimize")
    # Optimizing the objective function
    study.optimize(func=objective, n_trials=50, n_jobs=-1)

    # Log the best parameters
    mlflow.log_params(study.best_params)
    # Log the best MAPE
    mlflow.log_metric("best_mape", study.best_value)

[I 2025-03-22 00:00:26,855] A new study created in memory with name: model_selection


🏃 View run funny-jay-544 at: https://dagshub.com/SushrutGaikwad/taxi-demand-prediction.mlflow/#/experiments/0/runs/fb959430774c4fb38c2033746868ee76
🧪 View experiment at: https://dagshub.com/SushrutGaikwad/taxi-demand-prediction.mlflow/#/experiments/0
🏃 View run intelligent-dog-10 at: https://dagshub.com/SushrutGaikwad/taxi-demand-prediction.mlflow/#/experiments/0/runs/612598c975594e3e87f9f2168435a07c
🧪 View experiment at: https://dagshub.com/SushrutGaikwad/taxi-demand-prediction.mlflow/#/experiments/0
🏃 View run defiant-quail-89 at: https://dagshub.com/SushrutGaikwad/taxi-demand-prediction.mlflow/#/experiments/0/runs/8c362233766a4d6b8efea5b0eb4ffa81
🧪 View experiment at: https://dagshub.com/SushrutGaikwad/taxi-demand-prediction.mlflow/#/experiments/0
🏃 View run fortunate-ox-569 at: https://dagshub.com/SushrutGaikwad/taxi-demand-prediction.mlflow/#/experiments/0/runs/851df5b430864fdd95b6dcd9578f7a27
🧪 View experiment at: https://dagshub.com/SushrutGaikwad/taxi-demand-prediction.mlflow/#

[I 2025-03-22 00:01:13,829] Trial 3 finished with value: 0.07934790285463529 and parameters: {'model_name': 'LR'}. Best is trial 3 with value: 0.07934790285463529.


🏃 View run auspicious-gnat-418 at: https://dagshub.com/SushrutGaikwad/taxi-demand-prediction.mlflow/#/experiments/0/runs/2d118ee3b4ea4765811e673d89815b6b
🧪 View experiment at: https://dagshub.com/SushrutGaikwad/taxi-demand-prediction.mlflow/#/experiments/0


[I 2025-03-22 00:01:15,839] Trial 2 finished with value: 0.10808084905147552 and parameters: {'model_name': 'XGBR', 'n_estimators_xgb': 100, 'learning_rate_xgb': 0.09782463377770757, 'max_depth_xgb': 8}. Best is trial 3 with value: 0.07934790285463529.
[I 2025-03-22 00:01:17,817] Trial 12 finished with value: 0.07934790285463529 and parameters: {'model_name': 'LR'}. Best is trial 3 with value: 0.07934790285463529.
[I 2025-03-22 00:01:21,900] Trial 1 finished with value: 1.048230528831482 and parameters: {'model_name': 'XGBR', 'n_estimators_xgb': 30, 'learning_rate_xgb': 0.06124248879280332, 'max_depth_xgb': 9}. Best is trial 3 with value: 0.07934790285463529.
[I 2025-03-22 00:01:22,826] Trial 19 finished with value: 0.07934790285463529 and parameters: {'model_name': 'LR'}. Best is trial 3 with value: 0.07934790285463529.
[I 2025-03-22 00:01:24,848] Trial 5 finished with value: 0.07934790285463529 and parameters: {'model_name': 'LR'}. Best is trial 3 with value: 0.07934790285463529.


...

🏃 View run flawless-conch-63 at: https://dagshub.com/SushrutGaikwad/taxi-demand-prediction.mlflow/#/experiments/0/runs/c0650b3beb90456d80dd38201f99b386
🧪 View experiment at: https://dagshub.com/SushrutGaikwad/taxi-demand-prediction.mlflow/#/experiments/0
🏃 View run gentle-steed-541 at: https://dagshub.com/SushrutGaikwad/taxi-demand-prediction.mlflow/#/experiments/0/runs/f1b13896dded4e079a024d48322cbe76
🧪 View experiment at: https://dagshub.com/SushrutGaikwad/taxi-demand-prediction.mlflow/#/experiments/0


[I 2025-03-22 00:05:07,849] Trial 6 finished with value: 0.07934790285463529 and parameters: {'model_name': 'LR'}. Best is trial 3 with value: 0.07934790285463529.
[I 2025-03-22 00:05:08,822] Trial 4 finished with value: 0.07934790285463529 and parameters: {'model_name': 'LR'}. Best is trial 3 with value: 0.07934790285463529.
[I 2025-03-22 00:05:09,848] Trial 20 finished with value: 0.07934790285463529 and parameters: {'model_name': 'LR'}. Best is trial 3 with value: 0.07934790285463529.
[I 2025-03-22 00:05:17,842] Trial 7 finished with value: 0.3463017707490639 and parameters: {'model_name': 'RF', 'n_estimators_rf': 60, 'max_depth_rf': 4}. Best is trial 3 with value: 0.07934790285463529.
[I 2025-03-22 00:05:50,000] Trial 18 finished with value: 3.4244388762626796 and parameters: {'model_name': 'GBR', 'n_estimators_gb': 80, 'learning_rate_gb': 0.00883764074057375}. Best is trial 3 with value: 0.07934790285463529.


🏃 View run stylish-dog-341 at: https://dagshub.com/SushrutGaikwad/taxi-demand-prediction.mlflow/#/experiments/0/runs/ffba01f1053c438bac8adab03d9066be
🧪 View experiment at: https://dagshub.com/SushrutGaikwad/taxi-demand-prediction.mlflow/#/experiments/0


[I 2025-03-22 00:06:07,912] Trial 13 finished with value: 6.499630903026681 and parameters: {'model_name': 'GBR', 'n_estimators_gb': 30, 'learning_rate_gb': 0.00042106519411305206}. Best is trial 3 with value: 0.07934790285463529.


🏃 View run inquisitive-ram-287 at: https://dagshub.com/SushrutGaikwad/taxi-demand-prediction.mlflow/#/experiments/0/runs/08ebc0d1b437408fbd6079fffec15a33
🧪 View experiment at: https://dagshub.com/SushrutGaikwad/taxi-demand-prediction.mlflow/#/experiments/0
🏃 View run best_model at: https://dagshub.com/SushrutGaikwad/taxi-demand-prediction.mlflow/#/experiments/0/runs/db9c6b4e16eb4806900f86e1524fb938
🧪 View experiment at: https://dagshub.com/SushrutGaikwad/taxi-demand-prediction.mlflow/#/experiments/0


In [18]:
# Best MAPE

print(f"The best MAPE is {study.best_value * 100:.2f}%.")

The best MAPE is 7.93%.


In [19]:
# Best model

study.best_params

{'model_name': 'LR'}

In [20]:
# Model counts during the optimization

study.trials_dataframe()["params_model_name"].value_counts()

params_model_name
LR      27
XGBR    10
GBR      7
RF       6
Name: count, dtype: int64

# Training the best model

In [29]:
regressor = LinearRegression()
regressor.fit(X_train_encoded, y_train)

# Predictions

y_pred_train = regressor.predict(X_train_encoded) 
y_pred_test = regressor.predict(X_test_encoded)

# MAPE

mape_train = mean_absolute_percentage_error(y_true=y_train, y_pred=y_pred_train)
mape_test = mean_absolute_percentage_error(y_true=y_test, y_pred=y_pred_test)

print(f"The training MAPE is {mape_train*100:.2f}%.")
print(f"The test MAPE is {mape_test*100:.2f}%.")

The training MAPE is 8.78%.
The test MAPE is 7.93%.


In [30]:
regressor.coef_

array([-2.33737604,  0.71512405, -0.55601505, -1.25311068, -3.20463231,
       -0.86685973, -2.79925402, -3.62516859,  0.41386463, -2.9376376 ,
       -1.97624678, -3.75050442,  0.51806283, -2.54033388, -2.43297463,
        0.47632075,  0.61254786, -4.7417372 , -2.03077217, -1.26960984,
       -4.03690273, -2.08863167, -1.0414428 ,  0.73561736, -0.99999442,
       -0.85944985, -2.43098478,  0.67112238,  0.57385071, -0.11719951,
       -0.28045898, -0.37180749, -0.5238324 , -0.4233113 , -0.34045774,
       -0.54170892, -0.36264553, -0.2493965 , -0.31905518,  2.4912456 ])

Let us try ridge regression, which is a regularized version of linear regression.

In [31]:
def tune_ridge(trial):
    # Hyperparameter space
    alpha = trial.suggest_float("alpha", 30, 100)
    
    # Model
    ridge = Ridge(alpha=alpha, random_state=42)
    
    # Training
    ridge.fit(X_train_encoded, y_train)
    
    # Predicting
    y_pred = ridge.predict(X_test_encoded)
    
    # MAPE
    mape = mean_absolute_percentage_error(y_true=y_test, y_pred=y_pred)

    return mape

In [32]:
# Create study

study = optuna.create_study(study_name="tune_ridge", direction="minimize")

[I 2025-03-22 00:30:08,781] A new study created in memory with name: tune_ridge


In [34]:
# optimize

study.optimize(func=tune_ridge, n_trials=100, n_jobs=-1, show_progress_bar=True)

  0%|          | 0/100 [00:00<?, ?it/s]

[I 2025-03-22 00:31:19,858] Trial 45 finished with value: 0.07918544977905692 and parameters: {'alpha': 83.68581987244933}. Best is trial 42 with value: 0.07916408447959469.
[I 2025-03-22 00:31:20,064] Trial 46 finished with value: 0.07918567126002436 and parameters: {'alpha': 83.52663433389492}. Best is trial 42 with value: 0.07916408447959469.
[I 2025-03-22 00:31:20,095] Trial 47 finished with value: 0.07918311756121663 and parameters: {'alpha': 85.37405241031513}. Best is trial 42 with value: 0.07916408447959469.
[I 2025-03-22 00:31:20,135] Trial 51 finished with value: 0.07918458695645363 and parameters: {'alpha': 84.30741244451377}. Best is trial 42 with value: 0.07916408447959469.
[I 2025-03-22 00:31:20,176] Trial 49 finished with value: 0.07918238302795762 and parameters: {'alpha': 85.9109455964816}. Best is trial 42 with value: 0.07916408447959469.
[I 2025-03-22 00:31:20,206] Trial 48 finished with value: 0.07918748666258128 and parameters: {'alpha': 82.23005242847847}. Best is

In [35]:
# Best parameters

study.best_params

{'alpha': 99.99555980666432}

In [37]:
# Best MAPE

print(f"The best MAPE using ridge regression is {study.best_value*100:.2f}%.")

The best MAPE using ridge regression is 7.92%.


So we will go with the linear regression model. We won't do any hyperparameter tuning for it as it does not have any hyperparameters.